In [3]:
import time
import pandas as pd
from pathlib import Path
from datetime import datetime
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.edge.options import Options

In [4]:
driver_path = 'msedgedriver.exe'
service = Service(driver_path)
driver = webdriver.Edge(service=service)